In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'utilizando dispositivo {device}')

utilizando dispositivo cuda


In [3]:
# transformação para o pre processamento das imagens
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(), # converter pra tensor
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)) #normalização para [-1, 1]
])

In [4]:
dataset = datasets.ImageFolder(root='PetImages', transform=transform)

In [5]:
# definir proporções entre treino e teste
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [6]:
# dividir dados aleatoriamente
train_data, test_data = random_split(dataset, [train_size, test_size])

In [7]:
# criar data loader
batch_size = 24
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle= False)

In [8]:
print(f'tamanho do conjunto de treino: {len(train_data)} imagens')
print(f'tamanho do conjunto de teste: {len(test_data)} imagens')

tamanho do conjunto de treino: 19998 imagens
tamanho do conjunto de teste: 5000 imagens


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2) # saida de duas classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16) # flatten
        x = self.relu(self.fc1(x)) 
        x = self.fc2(x) #Removemos Softmax porque CrossEntropyLoss já lida com isso
        return x

In [10]:
# criar a rede e mover pra GPU
modelo = CNN().to(device)

# função de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelo.parameters(), lr = 0.001)

In [11]:
# treino
def train(modelo, train_loader, criterion, optimizer, epochs = 10):
    modelo.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = modelo(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            print(f"Época {epoch + 1} Loss: {running_loss/len(train_loader):.4f}")


In [12]:
# teste
def test(modelo, test_loader):
    modelo.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = modelo(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f"Acurácia: {100 * correct / total:.2f}%")

In [13]:
# treinamento
train(modelo, train_loader, criterion, optimizer, epochs = 20)

Época 1 Loss: 0.0008
Época 1 Loss: 0.0017
Época 1 Loss: 0.0034
Época 1 Loss: 0.0043
Época 1 Loss: 0.0053
Época 1 Loss: 0.0061
Época 1 Loss: 0.0069
Época 1 Loss: 0.0078
Época 1 Loss: 0.0086
Época 1 Loss: 0.0094
Época 1 Loss: 0.0102
Época 1 Loss: 0.0111
Época 1 Loss: 0.0119
Época 1 Loss: 0.0127
Época 1 Loss: 0.0135
Época 1 Loss: 0.0144
Época 1 Loss: 0.0152
Época 1 Loss: 0.0160
Época 1 Loss: 0.0168
Época 1 Loss: 0.0176
Época 1 Loss: 0.0185
Época 1 Loss: 0.0193
Época 1 Loss: 0.0201
Época 1 Loss: 0.0209
Época 1 Loss: 0.0217
Época 1 Loss: 0.0226
Época 1 Loss: 0.0234
Época 1 Loss: 0.0242
Época 1 Loss: 0.0250
Época 1 Loss: 0.0258
Época 1 Loss: 0.0266
Época 1 Loss: 0.0275
Época 1 Loss: 0.0283
Época 1 Loss: 0.0291
Época 1 Loss: 0.0299
Época 1 Loss: 0.0307
Época 1 Loss: 0.0315
Época 1 Loss: 0.0324
Época 1 Loss: 0.0333
Época 1 Loss: 0.0342
Época 1 Loss: 0.0350
Época 1 Loss: 0.0358
Época 1 Loss: 0.0366
Época 1 Loss: 0.0374
Época 1 Loss: 0.0382
Época 1 Loss: 0.0390
Época 1 Loss: 0.0398
Época 1 Loss:

C:\Users\Mateus\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Época 1 Loss: 0.0694
Época 1 Loss: 0.0702
Época 1 Loss: 0.0710
Época 1 Loss: 0.0719
Época 1 Loss: 0.0727
Época 1 Loss: 0.0735
Época 1 Loss: 0.0743
Época 1 Loss: 0.0751
Época 1 Loss: 0.0759
Época 1 Loss: 0.0767
Época 1 Loss: 0.0776
Época 1 Loss: 0.0784
Época 1 Loss: 0.0792
Época 1 Loss: 0.0800
Época 1 Loss: 0.0808
Época 1 Loss: 0.0817
Época 1 Loss: 0.0826
Época 1 Loss: 0.0833
Época 1 Loss: 0.0841
Época 1 Loss: 0.0849
Época 1 Loss: 0.0857
Época 1 Loss: 0.0866
Época 1 Loss: 0.0874
Época 1 Loss: 0.0882
Época 1 Loss: 0.0890
Época 1 Loss: 0.0897
Época 1 Loss: 0.0905
Época 1 Loss: 0.0913
Época 1 Loss: 0.0921
Época 1 Loss: 0.0928
Época 1 Loss: 0.0935
Época 1 Loss: 0.0943
Época 1 Loss: 0.0953
Época 1 Loss: 0.0960
Época 1 Loss: 0.0967
Época 1 Loss: 0.0976
Época 1 Loss: 0.0984
Época 1 Loss: 0.0992
Época 1 Loss: 0.1001
Época 1 Loss: 0.1008
Época 1 Loss: 0.1016
Época 1 Loss: 0.1024
Época 1 Loss: 0.1032
Época 1 Loss: 0.1039
Época 1 Loss: 0.1046
Época 1 Loss: 0.1055
Época 1 Loss: 0.1062
Época 1 Loss:

In [14]:
# teste
test(modelo, test_loader)

Acurácia: 78.46%


In [15]:
torch.save(modelo.state_dict(), "modelo_cachorros_gatos.pth")
print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!
